In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install scikit-learn
%pip install matplotlib

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Define dataset paths
train_data_path = 'C:\\projects\\BloodNets\\blood_data\\train'
test_data_path = 'C:\\projects\\BloodNets\\blood_data\\test'

# Load datasets
train_data = datasets.ImageFolder(train_data_path, transform=transform)
test_data = datasets.ImageFolder(test_data_path, transform=transform)

In [5]:
# Define and initialize models
def initialize_model(model_name, num_classes):
    if model_name == 'shufflenet':
        model = models.shufflenet_v2_x1_0(pretrained=True)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    elif model_name == 'mobilenet':
        model = models.mobilenet_v2(pretrained=True)
        num_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_features, num_classes)
    elif model_name == 'darknet':
        from torchvision.models.detection import ssdlite320_mobilenet_v3_large
        model = ssdlite320_mobilenet_v3_large(pretrained=True)
        num_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = nn.Linear(num_features, num_classes)
    else:
        raise ValueError("Unsupported model name")
    return model

In [6]:
# Training function
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

In [7]:
# Testing function
def test(model, loader, device):
    model.eval()
    all_labels, all_preds = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    return np.array(all_labels), np.array(all_preds)

In [8]:
# Train base models and generate predictions for stacking
def train_and_predict(models_list, train_data, test_data):
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)
    
    base_predictions_train = []
    base_predictions_test = []

    for model in models_list:
        model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        for epoch in range(5):  # Adjust number of epochs as needed
            train_loss = train(model, train_loader, criterion, optimizer, device)
            print(f"Training {model.__class__.__name__} - Epoch {epoch + 1}/5 - Loss: {train_loss:.4f}")

        # Generate predictions
        train_labels, train_preds = test(model, train_loader, device)
        test_labels, test_preds = test(model, test_loader, device)
        
        base_predictions_train.append(train_preds)
        base_predictions_test.append(test_preds)
            # Convert predictions to arrays
    base_predictions_train = np.stack(base_predictions_train, axis=1)
    base_predictions_test = np.stack(base_predictions_test, axis=1)
    
    return base_predictions_train, base_predictions_test, train_labels, test_labels

In [10]:
# Define models
model_names = ['shufflenet', 'mobilenet']
models_list = [initialize_model(name, len(train_data.classes)) for name in model_names]

# Train models and get predictions
base_predictions_train, base_predictions_test, train_labels, test_labels = train_and_predict(models_list, train_data, test_data)

# Train meta-model
meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(base_predictions_train, train_labels)

# Predict with meta-model
meta_predictions = meta_model.predict(base_predictions_test)

Training ShuffleNetV2 - Epoch 1/5 - Loss: 0.5774
Training ShuffleNetV2 - Epoch 2/5 - Loss: 0.1506
Training ShuffleNetV2 - Epoch 3/5 - Loss: 0.0993
Training ShuffleNetV2 - Epoch 4/5 - Loss: 0.0757
Training ShuffleNetV2 - Epoch 5/5 - Loss: 0.0634
Training MobileNetV2 - Epoch 1/5 - Loss: 0.3030
Training MobileNetV2 - Epoch 2/5 - Loss: 0.1841
Training MobileNetV2 - Epoch 3/5 - Loss: 0.1312
Training MobileNetV2 - Epoch 4/5 - Loss: 0.1159
Training MobileNetV2 - Epoch 5/5 - Loss: 0.1101


In [12]:
# Evaluate meta-model
metrics = {
    'accuracy': accuracy_score(test_labels, meta_predictions),
    'precision': precision_score(test_labels, meta_predictions, average='weighted'),
    'recall': recall_score(test_labels, meta_predictions, average='weighted'),
    'f1_score': f1_score(test_labels, meta_predictions, average='weighted')
}

print("Stacking Ensemble Metrics")
print(f"Accuracy: {metrics['accuracy']*100:.4f}")
print(f"Precision: {metrics['precision']*100:.4f}")
print(f"Recall: {metrics['recall']*100:.4f}")
print(f"F1 Score: {metrics['f1_score']*100:.4f}")

Stacking Ensemble Metrics
Accuracy: 95.9983
Precision: 96.1066
Recall: 95.9983
F1 Score: 95.8711
